-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Feature Store

The <a href="https://docs.databricks.com/applications/machine-learning/feature-store.html" target="_blank">Databricks Feature Store</a> is a centralized repository of features. It enables feature sharing and discovery across your organization and also ensures that the same feature computation code is used for model training and inference.

Check out Feature Store Python API documentation <a href="https://docs.databricks.com/dev-tools/api/python/latest/index.html#feature-store-python-api-reference" target="_blank">here</a>.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you will:<br>
 - **Build a feature store with the Databricks Feature Store**
 - **Update feature tables**
 - **Perform batch scoring**

In [0]:
%run "./Includes/Classroom-Setup"

Defining courseware-specific utility methods...

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/scalable-machine-learning-with-apache-spark/v01/

Your dataset directory is
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets

Skipping install of existing dataset.

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, lit, expr, rand
import uuid

from databricks import feature_store
from pyspark.sql.types import StringType, DoubleType
from databricks.feature_store import feature_table, FeatureLookup
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

#### Let's load in our data and generate a unique ID for each listing.
- The **`index`** column will serve as the "key" of the feature table and used to lookup features.

In [0]:
file_path = f"{datasets_dir}/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"
airbnb_df = spark.read.format("delta").load(file_path).coalesce(1).withColumn("index", monotonically_increasing_id())
display(airbnb_df)

host_is_superhost,cancellation_policy,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price,bedrooms_na,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na,index
t,moderate,f,2.0,South of Market,37.77818,-122.41444,Apartment,Entire home/apt,5.0,1.0,2.0,3.0,Real Bed,1.0,14.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
f,strict_14_with_grace_period,t,6.0,Western Addition,37.77905,-122.43992,House,Private room,6.0,1.0,2.0,3.0,Real Bed,1.0,36.0,93.0,9.0,10.0,10.0,10.0,9.0,8.0,425.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
t,strict_14_with_grace_period,f,2.0,West of Twin Peaks,37.73313,-122.46054,House,Private room,4.0,1.0,2.0,3.0,Real Bed,3.0,207.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,129.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
f,flexible,t,14.0,Western Addition,37.78172,-122.43861,House,Private room,2.0,1.0,1.0,1.0,Real Bed,30.0,1.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
f,strict_14_with_grace_period,f,44.0,Mission,37.75825,-122.41512,Apartment,Entire home/apt,4.0,1.0,2.0,2.0,Real Bed,30.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,156.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4
f,strict_14_with_grace_period,t,1.0,Inner Sunset,37.7491,-122.46782,Apartment,Entire home/apt,3.0,1.0,1.0,1.0,Real Bed,1.0,29.0,95.0,10.0,10.0,10.0,10.0,9.0,9.0,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
t,moderate,f,2.0,Castro/Upper Market,37.76925,-122.43077,Apartment,Entire home/apt,5.0,1.0,2.0,2.0,Real Bed,3.0,17.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,290.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
f,super_strict_60,f,16.0,Financial District,37.78931,-122.40231,Serviced apartment,Entire home/apt,6.0,2.5,2.0,3.0,Real Bed,1.0,2.0,100.0,10.0,10.0,10.0,10.0,10.0,9.0,1300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
t,moderate,t,2.0,Castro/Upper Market,37.76077,-122.43786,Apartment,Entire home/apt,3.0,1.0,1.0,1.0,Real Bed,30.0,10.0,98.0,10.0,9.0,10.0,10.0,10.0,10.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
f,moderate,t,1.0,Outer Sunset,37.75359,-122.48137,Apartment,Entire home/apt,4.0,1.0,2.0,3.0,Real Bed,4.0,38.0,99.0,10.0,10.0,10.0,10.0,9.0,10.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


#### Create a new database and unique table name (in case you re-run the notebook multiple times)

In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {cleaned_username}")
table_name = f"{cleaned_username}.airbnb_" + str(uuid.uuid4())[:6]
print(table_name)

manujkumar_joshi.airbnb_6cfa38

#### Let's start creating a <a href="https://docs.databricks.com/applications/machine-learning/feature-store.html#create-a-feature-table-in-databricks-feature-store" target="_blank">Feature Store Client</a> so we can populate our feature store.

In [0]:
fs = feature_store.FeatureStoreClient()

help(fs.create_table)  ##documentation

Help on method create_table in module databricks.feature_store.client:

create_table(name: str, primary_keys: Union[str, List[str]], df: Union[pyspark.sql.dataframe.DataFrame, NoneType] = None, *, timestamp_keys: Union[str, List[str], NoneType] = None, partition_columns: Union[str, List[str], NoneType] = None, schema: Union[pyspark.sql.types.StructType, NoneType] = None, description: Union[str, NoneType] = None, **kwargs) -> databricks.feature_store.entities.feature_table.FeatureTable method of databricks.feature_store.client.FeatureStoreClient instance
 Create and return a feature table with the given name and primary keys.
 
 The returned feature table has the given name and primary keys.
 Uses the provided ``schema`` or the inferred schema
 of the provided ``df``. If ``df`` is provided, this data will be saved in
 a Delta table. Supported data types for features are: ``IntegerType``, ``LongType``,
 ``FloatType``, ``DoubleType``, ``StringType``, ``BooleanType``, ``DateType``,
 ``TimestampType``, ``ShortType``, ``ArrayType``, ``MapType``, and ``BinaryType``,
 and ``DecimalType``.
 
 :param name: A feature table name of the form ``<database_name>.<table_name>``,
 for example ``dev.user_features``.
 :param primary_keys: The feature table's primary keys. If multiple columns are required,
 specify a list of column names, for example ``['customer_id', 'region']``.
 :param df: Data to insert into this feature table. The schema of
 ``df`` will be used as the feature table schema.
 :param timestamp_keys: Columns containing the event time associated with feature value.
 Timestamp keys and primary keys of the feature table uniquely identify the feature value
 for an entity at a point in time.
 
 
 .. note::
 
 Experimental: This argument may change or be removed in
 a future release without warning.
 
 :param partition_columns: Columns used to partition the feature table. If a list is
 provided, column ordering in the list will be used for partitioning.
 
 .. Note:: When choosing partition columns for your feature table, use columns that do
 not have a high cardinality. An ideal strategy would be such that you
 expect data in each partition to be at least 1 GB.
 The most commonly used partition column is a ``date``.
 
 Additional info: `Choosing the right partition columns for Delta tables
 <https://bit.ly/3ueXsjv>`_
 :param schema: Feature table schema. Either ``schema`` or ``df`` must be provided.
 :param description: Description of the feature table.
 
 :Other Parameters:
 * **path** (``Optional[str]``) --
 Path in a supported filesystem. Defaults to the database location.

### Create Feature Table

Next, we can create the Feature Table using the **`create_table`** method.

This method takes a few parameters as inputs:
* **`name`**- A feature table name of the form **`<database_name>.<table_name>`**
* **`primary_keys`**- The primary key(s). If multiple columns are required, specify a list of column names.
* **`df`**- Data to insert into this feature table.  The schema of **`features_df`** will be used as the feature table schema.
* **`schema`**- Feature table schema. Note that either **`schema`** or **`features_df`** must be provided.
* **`description`**- Description of the feature table
* **`partition_columns`**- Column(s) used to partition the feature table.

- First we define a **`@feature_table`** that simply selects some numeric information from the data. This will become one feature store table when written later. 
- A **`@feature_table`** is really just a function that computes a DataFrame defining the features in the table, from a source 'raw' DataFrame. It can be called directly for testing; by itself, it does not persist or publish features.

In [0]:
## select numeric features and aggregate the review scores, exclude target column "price"
numeric_cols = [x.name for x in airbnb_df.schema.fields if (x.dataType == DoubleType()) and (x.name != "price")]

@feature_table
def select_numeric_features(data):
    return data.select(["index"] + numeric_cols)

numeric_features_df = select_numeric_features(airbnb_df)
display(numeric_features_df)

index,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,bedrooms_na,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na
0,2.0,37.77818,-122.41444,5.0,1.0,2.0,3.0,1.0,14.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.0,37.77905,-122.43992,6.0,1.0,2.0,3.0,1.0,36.0,93.0,9.0,10.0,10.0,10.0,9.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,37.73313,-122.46054,4.0,1.0,2.0,3.0,3.0,207.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14.0,37.78172,-122.43861,2.0,1.0,1.0,1.0,30.0,1.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,44.0,37.75825,-122.41512,4.0,1.0,2.0,2.0,30.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,37.7491,-122.46782,3.0,1.0,1.0,1.0,1.0,29.0,95.0,10.0,10.0,10.0,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,37.76925,-122.43077,5.0,1.0,2.0,2.0,3.0,17.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,16.0,37.78931,-122.40231,6.0,2.5,2.0,3.0,1.0,2.0,100.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2.0,37.76077,-122.43786,3.0,1.0,1.0,1.0,30.0,10.0,98.0,10.0,9.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,37.75359,-122.48137,4.0,1.0,2.0,3.0,4.0,38.0,99.0,10.0,10.0,10.0,10.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
fs.create_table(
    name=table_name,
    primary_keys=["index"],
    df=numeric_features_df,
    schema=numeric_features_df.schema,
    description="Numeric features of airbnb data"
)

2022/09/03 06:17:37 INFO databricks.feature_store.client: Created feature table 'manujkumar_joshi.airbnb_6cfa38'.
Out[12]: <FeatureTable: keys=['index']>

Alternatively, you can **`create_table`** with schema only (without **`df`**), and populate data to the feature table with **`fs.write_table`**. **`fs.write_table`** supports both **`overwrite`** and **`merge`** modes.

Example:

**`
fs.create_table(
    name=table_name,
    primary_keys=["index"],
    schema=numeric_features_df.schema,
    description="Original Airbnb data"
)
fs.write_table(
    name=table_name,
    df=numeric_features_df,
    mode="overwrite"
)
`**

Now let's explore the UI and see how it tracks the tables that we created. Navigate to the UI by first ensuring that you are in the Machine Learning workspace, and then clicking on the Feature Store icon on the bottom-left of the navigation bar.


<img src="https://files.training.databricks.com/images/mlflow/FS_Nav.png" alt="step12" width="150"/>

In this screenshot, we can see the feature table that we created.
<br>
<br>
Note the section of **`Producers`**. This section indicates which notebook produces the feature table.
<br>
<br>
<img src="https://s3.us-west-2.amazonaws.com/files.training.databricks.com/images/mlflow/fs_details+(1).png" alt="step12" width="1000"/>

#### We can also look at the metadata of the feature store via the FeatureStore client by using **`get_table()`**.

In [0]:
fs.get_table(table_name).path_data_sources

Out[13]: ['dbfs:REDACTED_LOCAL_PART@celebaltech.com/dbacademy/machine_learning/datasets/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta']

In [0]:
len(numeric_features_df.columns)

Out[14]: 27

In [0]:
fs.get_table(table_name).description

Out[15]: 'Numeric features of airbnb data'

In [0]:
len(fs.get_table('rahul_bisht.airbnb_8ac195').features)  ## created one extra feature with feature engineering

Out[16]: 28

## Train a model with feature store

The prediction target **`price`** should NOT BE included as a feature in the registered feature table.

Further, there may be other information that _can_ be supplied at inference time, but does not make sense to consider a feature to _look up_. 

In this (fictional) example, we made up a feature **`score_diff_from_last_month`**. It is a feature generated at inference time and used in training as well.

In [0]:
## inference data -- index (key), price (target) and a online feature (make up a fictional column - diff of review score in a month) 
inference_data_df = airbnb_df.select("index", "price", (rand() * 0.5-0.25).alias("score_diff_from_last_month"))
display(inference_data_df)

index,price,score_diff_from_last_month
0,199.0,0.028644289891968877
1,425.0,-0.12163087656749283
2,129.0,0.18325758965055416
3,55.0,0.23590195626009047
4,156.0,0.17361598024892883
5,125.0,0.1063999722352259
6,290.0,0.191199816939366
7,1300.0,0.22258507920559556
8,250.0,-0.24189307971765223
9,180.0,0.049065859139404866


##### Build a training dataset that will use the indicated "key" to lookup features from the feature table and also the online feature **`score_diff_from_last_month`**. We will use <a href="https://docs.databricks.com/dev-tools/api/python/latest/feature-store/databricks.feature_store.entities.feature_lookup.html" target="_blank">FeatureLookup</a> and if you specify no features, it will return all of them except the primary key.

In [0]:
def load_data(table_name, lookup_key):
    model_feature_lookups = [FeatureLookup(table_name=table_name, lookup_key=lookup_key)]

    # fs.create_training_set will look up features in model_feature_lookups with matched key from inference_data_df
    training_set = fs.create_training_set(inference_data_df, model_feature_lookups, label="price", exclude_columns="index")
    training_pd = training_set.load_df().toPandas()

    # Create train and test datasets
    X = training_pd.drop("price", axis=1)
    y = training_pd["price"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, training_set

X_train, X_test, y_train, y_test, training_set = load_data(table_name, "index")
X_train.head()

Out[18]:

,score_diff_from_last_month,accommodates,bathrooms,bathrooms_na,bedrooms,bedrooms_na,beds,beds_na,host_total_listings_count,latitude,longitude,minimum_nights,number_of_reviews,review_scores_accuracy,review_scores_accuracy_na,review_scores_checkin,review_scores_checkin_na,review_scores_cleanliness,review_scores_cleanliness_na,review_scores_communication,review_scores_communication_na,review_scores_location,review_scores_location_na,review_scores_rating,review_scores_rating_na,review_scores_value,review_scores_value_na
59,-0.016515,3.0,1.0,0.0,1.0,0.0,2.0,0.0,2.0,37.74994,-122.41452,1.0,266.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,98.0,0.0,10.0,0.0
3717,-0.096418,4.0,1.0,0.0,2.0,0.0,3.0,0.0,2.0,37.76159,-122.44491,2.0,0.0,10.0,1.0,10.0,1.0,10.0,1.0,10.0,1.0,10.0,1.0,98.0,1.0,10.0,1.0
787,-0.249402,2.0,1.5,0.0,1.0,0.0,1.0,0.0,3.0,37.75011,-122.41680,2.0,50.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,96.0,0.0,9.0,0.0
2540,-0.197294,6.0,2.0,0.0,3.0,0.0,3.0,0.0,1.0,37.80031,-122.43142,30.0,4.0,10.0,0.0,10.0,0.0,10.0,0.0,9.0,0.0,10.0,0.0,95.0,0.0,9.0,0.0
6225,-0.190737,2.0,1.0,0.0,1.0,0.0,1.0,0.0,33.0,37.79669,-122.41223,30.0,0.0,10.0,1.0,10.0,1.0,10.0,1.0,10.0,1.0,10.0,1.0,98.0,1.0,10.0,1.0


- Train a **RandomForestRegressor** model and log the model with the Feature Store. 
- An MLflow run is started to track the autologged components as well as the Feature Store logged model. 
- However, we will disable the MLflow model autologging as the model will be explicitly logged via the Feature Store.

NOTE: This is an overly simplistic example, used solely for demo purposes.

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()

try:
    client.delete_registered_model(f"feature_store_airbnb_{cleaned_username}") # Deleting model if already created
except:
    None

In [0]:
# Disable model autologging and instead log explicitly via the FeatureStore
mlflow.sklearn.autolog(log_models=False)

def train_model(X_train, X_test, y_train, y_test, training_set, fs):
    ## fit and log model
    with mlflow.start_run() as run:

        rf = RandomForestRegressor(max_depth=3, n_estimators=20, random_state=42)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        mlflow.log_metric("test_mse", mean_squared_error(y_test, y_pred))
        mlflow.log_metric("test_r2_score", r2_score(y_test, y_pred))

        fs.log_model(
            model=rf,
            artifact_path="feature-store-model",
            flavor=mlflow.sklearn,
            training_set=training_set,
            registered_model_name=f"feature_store_airbnb_{cleaned_username}",
            input_example=X_train[:5],
            signature=infer_signature(X_train, y_train)
        )

train_model(X_train, X_test, y_train, y_test, training_set, fs)

Successfully registered model 'feature_store_airbnb_manujkumar_joshi'.
2022/09/03 06:17:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: feature_store_airbnb_manujkumar_joshi, version 1
Created version '1' of model 'feature_store_airbnb_manujkumar_joshi'.

Now, view the run from MLflow UI. You can find the model parameters logged with MLflow autolog.
<br>
<br>
<img src="https://files.training.databricks.com/images/mlflow/fs_log_model_mlflow_params.png" alt="step12" width="1000"/>

In addition, note the artifacts saved for the run, it saves two model artifacts:
 - **model** : raw sklearn flavor model - logged from mlflow autolog
 - **feature_store_model** : packaged feature store model that can be used directly for batch scoring - logged from **`fs.log_model`**
<br>
<br>

<img src="https://files.training.databricks.com/images/301/updated_feature_store_9_1.png" alt="step12" width="1000"/>

#### The **`feature_store_model`** is registered in the MLflow model registry as well. You can find it in **`Models`** page. It is also logged at the feature store page, indicating which features in the feature table are used for the model. We will exam feature/model lineage through the UI together later.

## Feature Store Batch Scoring

Apply a feature store registered MLflow model to features with **`score_batch`**. Input data only need the key column **`index`** and online feature **`score_diff_from_last_month`**. Everything else is looked up.

In [0]:
display(inference_data_df)

index,price,score_diff_from_last_month
0,199.0,0.028644289891968877
1,425.0,-0.12163087656749283
2,129.0,0.18325758965055416
3,55.0,0.23590195626009047
4,156.0,0.17361598024892883
5,125.0,0.1063999722352259
6,290.0,0.191199816939366
7,1300.0,0.22258507920559556
8,250.0,-0.24189307971765223
9,180.0,0.049065859139404866


In [0]:
## For sake of simplicity, we will just predict on the same inference_data_df
batch_input_df = inference_data_df.drop("price") # Exclude true label
predictions_df = fs.score_batch(f"models:/feature_store_airbnb_{cleaned_username}/1", 
                                  batch_input_df, result_type="double")
display(predictions_df)

index,score_diff_from_last_month,accommodates,bathrooms,bathrooms_na,bedrooms,bedrooms_na,beds,beds_na,host_total_listings_count,latitude,longitude,minimum_nights,number_of_reviews,review_scores_accuracy,review_scores_accuracy_na,review_scores_checkin,review_scores_checkin_na,review_scores_cleanliness,review_scores_cleanliness_na,review_scores_communication,review_scores_communication_na,review_scores_location,review_scores_location_na,review_scores_rating,review_scores_rating_na,review_scores_value,review_scores_value_na,prediction
0,0.028644289891968877,5.0,1.0,0.0,2.0,0.0,3.0,0.0,2.0,37.77818,-122.41444,1.0,14.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,100.0,0.0,10.0,0.0,296.2210608410127
1,-0.12163087656749283,6.0,1.0,0.0,2.0,0.0,3.0,0.0,6.0,37.77905,-122.43992,1.0,36.0,9.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,9.0,0.0,93.0,0.0,8.0,0.0,315.0267675650031
2,0.18325758965055416,4.0,1.0,0.0,2.0,0.0,3.0,0.0,2.0,37.73313,-122.46054,3.0,207.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,97.0,0.0,10.0,0.0,267.8910390147604
3,0.23590195626009047,2.0,1.0,0.0,1.0,0.0,1.0,0.0,14.0,37.78172,-122.43861,30.0,1.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,100.0,0.0,10.0,0.0,148.2736997545472
4,0.17361598024892883,4.0,1.0,0.0,2.0,0.0,2.0,0.0,44.0,37.75825,-122.41512,30.0,0.0,10.0,1.0,10.0,1.0,10.0,1.0,10.0,1.0,10.0,1.0,98.0,1.0,10.0,1.0,264.1745078090866
5,0.1063999722352259,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,37.7491,-122.46782,1.0,29.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,9.0,0.0,95.0,0.0,9.0,0.0,147.82111767169704
6,0.191199816939366,5.0,1.0,0.0,2.0,0.0,2.0,0.0,2.0,37.76925,-122.43077,3.0,17.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,98.0,0.0,10.0,0.0,296.2210608410127
7,0.22258507920559556,6.0,2.5,0.0,2.0,0.0,3.0,0.0,16.0,37.78931,-122.40231,1.0,2.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,100.0,0.0,9.0,0.0,360.96156858915435
8,-0.24189307971765223,3.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,37.76077,-122.43786,30.0,10.0,10.0,0.0,10.0,0.0,9.0,0.0,10.0,0.0,10.0,0.0,98.0,0.0,10.0,0.0,148.53202506201347
9,0.049065859139404866,4.0,1.0,0.0,2.0,0.0,3.0,0.0,1.0,37.75359,-122.48137,4.0,38.0,10.0,0.0,10.0,0.0,10.0,0.0,10.0,0.0,9.0,0.0,99.0,0.0,10.0,0.0,267.8910390147604


### Overwrite feature table
Lastly, we'll condense some of the review columns and update the feature table: we'll do this by calculating the average review score for each listing.

In [0]:
## select numeric features and aggregate the review scores
review_columns = ["review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", 
                 "review_scores_communication", "review_scores_location", "review_scores_value"]
@feature_table
def select_numeric_features(data):
    result = (data.select(["index"] + numeric_cols)
              .withColumn("average_review_score", expr("+".join(review_columns)) / lit(len(review_columns)))
              .drop(*review_columns)
             )
    return result

condensed_review_df = select_numeric_features(airbnb_df)
display(condensed_review_df)

index,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,bedrooms_na,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na,average_review_score
0,2.0,37.77818,-122.41444,5.0,1.0,2.0,3.0,1.0,14.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
1,6.0,37.77905,-122.43992,6.0,1.0,2.0,3.0,1.0,36.0,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.333333333333334
2,2.0,37.73313,-122.46054,4.0,1.0,2.0,3.0,3.0,207.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
3,14.0,37.78172,-122.43861,2.0,1.0,1.0,1.0,30.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4,44.0,37.75825,-122.41512,4.0,1.0,2.0,2.0,30.0,0.0,98.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0
5,1.0,37.7491,-122.46782,3.0,1.0,1.0,1.0,1.0,29.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.666666666666666
6,2.0,37.76925,-122.43077,5.0,1.0,2.0,2.0,3.0,17.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
7,16.0,37.78931,-122.40231,6.0,2.5,2.0,3.0,1.0,2.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.833333333333334
8,2.0,37.76077,-122.43786,3.0,1.0,1.0,1.0,30.0,10.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.833333333333334
9,1.0,37.75359,-122.48137,4.0,1.0,2.0,3.0,4.0,38.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.833333333333334


In [0]:
airbnb_df.toPandas().head()

Out[24]:

,host_is_superhost,cancellation_policy,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price,bedrooms_na,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na,index
0,t,moderate,f,2.0,South of Market,37.77818,-122.41444,Apartment,Entire home/apt,5.0,1.0,2.0,3.0,Real Bed,1.0,14.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,f,strict_14_with_grace_period,t,6.0,Western Addition,37.77905,-122.43992,House,Private room,6.0,1.0,2.0,3.0,Real Bed,1.0,36.0,93.0,9.0,10.0,10.0,10.0,9.0,8.0,425.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,t,strict_14_with_grace_period,f,2.0,West of Twin Peaks,37.73313,-122.46054,House,Private room,4.0,1.0,2.0,3.0,Real Bed,3.0,207.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,129.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,f,flexible,t,14.0,Western Addition,37.78172,-122.43861,House,Private room,2.0,1.0,1.0,1.0,Real Bed,30.0,1.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,f,strict_14_with_grace_period,f,44.0,Mission,37.75825,-122.41512,Apartment,Entire home/apt,4.0,1.0,2.0,2.0,Real Bed,30.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,156.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4


#### Let's now drop those features using **`overwrite`**.

In [0]:
fs.write_table(
    name=table_name,
    df=condensed_review_df,
    mode="overwrite"
)

### Explore the feature permission, lineage and freshness from Feature Store UI

#### On the UI, we can see that:
- A new column has been added to the feature list;
- Columns that we deleted are also still present. However, the deleted features will have **`null`** as their values when we read in the table;
- The "Models" column are populated, listing models use the features from the table
- The last column **`Notebooks`** are populated. This column indicates which notebooks consume the features in the feature table.

<img src="https://files.training.databricks.com/images/mlflow/feature_store_list.png" alt="step12" width="800"/>

Now, let's read in the feature data from the Feature Store. By default, **`fs.read_table()`** reads in the latest version of the feature table. To read in the specific version of feature table, you can optionally specify the argument **`as_of_delta_timestamp`** by passing a date in a timestamp format or string.


Note that the values of the deleted columns have been replaced by **`null`**.

In [0]:
# Displays most recent table
display(fs.read_table(name=table_name))

index,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,bedrooms_na,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na,average_review_score
0,2.0,37.77818,-122.41444,5.0,1.0,2.0,3.0,1.0,14.0,100.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
1,6.0,37.77905,-122.43992,6.0,1.0,2.0,3.0,1.0,36.0,93.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.333333333333334
2,2.0,37.73313,-122.46054,4.0,1.0,2.0,3.0,3.0,207.0,97.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
3,14.0,37.78172,-122.43861,2.0,1.0,1.0,1.0,30.0,1.0,100.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4,44.0,37.75825,-122.41512,4.0,1.0,2.0,2.0,30.0,0.0,98.0,null,null,null,null,null,null,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0
5,1.0,37.7491,-122.46782,3.0,1.0,1.0,1.0,1.0,29.0,95.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.666666666666666
6,2.0,37.76925,-122.43077,5.0,1.0,2.0,2.0,3.0,17.0,98.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
7,16.0,37.78931,-122.40231,6.0,2.5,2.0,3.0,1.0,2.0,100.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.833333333333334
8,2.0,37.76077,-122.43786,3.0,1.0,1.0,1.0,30.0,10.0,98.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.833333333333334
9,1.0,37.75359,-122.48137,4.0,1.0,2.0,3.0,4.0,38.0,99.0,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.833333333333334


If you need to use the features for real-time serving, you can publish your features to an <a href="https://docs.databricks.com/applications/machine-learning/feature-store.html#publish-features-to-an-online-feature-store" target="_blank">online store</a>.

We can perform control who has permissions to the feature table on the UI.

To delete the table, use the **`delete`** button on the UI. **You need to delete the delta table from database as well.**
<img src="https://s3.us-west-2.amazonaws.com/files.training.databricks.com/images/mlflow/fs_permissions+(1).png" alt="step12" width="700"/>

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>